# Advanced research lab report - Bayesian optimization for more effective search of the thermoelectic material space

# Introduction

The space of possible thermoelectric materials is too large to be fully explored experimentally, or even using high-throughput computational techniques. Machine learning, specifically, Bayesian optimization (BO) represents a new tool that can help scientists explore the material space and discover new high-performance materials more efficiently. 

BO is usually characterized by an iteretive loop:
    1. Select an initial data set
    2. Train a surrogate model to imitate the relationship between the features and the target property
    3. Using an appropriate acquisition function, find a point in the (bounded) feature space which has the best balance of exploration and exploitation
    4. Evaluate the target property at the chosen features
    5. Append the new point to the data set, repeat until a set number of iterations is achieved

This work explores the use of different methods of obtaining the surrogate models, mainly comparing the typical gaussian process (GP) to a probabilistic Bayesian neural network (pBNN). 

BO is the workflow of choice for the analysis of complicated, multi-dimentional data. A thermoelectic materials data set was chosen, following the work (Y.-F. Lim, C. K. Ng, K. Hippalgaonkar, 2021). The data has been obtained from high-throughput simulations for many materials at different temperatures. The target property is the power factor and the features space consists of e.g. mean covalent radius, temperature, doping or electronegativity.

To test the different surrogates the BO will be repeated for each method and the averages compared. Point 4 of the BO loop is the one that in practice takes the longest, sometimes even months. To this extent, a neural network (NN) is trained on the very large data set to model the relationship between features and the target. It is used to act as a predictive oracle which replaces the experimental (computational) evaluation with a simple evaluation of the NN. 

The best surrogate model, which achieves the highest power-factor in the fewest number of iterations is established to be the pBNN.



# Code


Importing all the important libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.utils import shuffle

from skopt import dummy_minimize
from skopt import forest_minimize
from skopt import gbrt_minimize
from skopt.optimizer import base_minimize
from NeuralEnsemble import NeuralEnsembleRegressor
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import ConstantKernel, Matern

import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
device = 'cuda:0'
dtype = torch.float

## Data preperation

The data set has been prepared before by Hippalgaonkar et al., 2021. The same workflow is used here to obtain reasonable test and train sets. Transformations are performed both on the feature space and the target to make the training of the NN more effective.

In [ ]:
data = pd.read_excel('./cubic.xlsx')
pipeline = Pipeline(steps=[('Scaler', StandardScaler()), ('Yeo-Johnson', PowerTransformer(method='yeo-johnson'))])

def data_prep(data):
    #data.hist(figsize =(15,15))
    #plt.tight_layout()
    #plt.show()
    data['Power Factor'] = data['Power Factor'] * 1e-23
    drop = ['index', 'nelements', 'n', 'p', 'nsites', 'direct', 'indirect']
    df_drop = data.drop(drop, axis=1)
    df_drop['Cbrt_PF'] = np.cbrt(df_drop['Power Factor'])
    X = df_drop.drop(['Power Factor', 'Cbrt_PF'], axis=1)
    Y = df_drop['Cbrt_PF']

    column = X.columns 
    X, Y = shuffle(X,Y)
    X = pipeline.fit_transform(X)
    X = pd.DataFrame(X)
    X.columns = column    
    #Y = Y.values
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.1, random_state=1)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_prep(data)
columns = X_train.columns 

## Preparation of predictive oracle

Plotting function for validation of regression

In [ ]:
# Plot prediction vs true values for training and test sets - interpolation
def plot(regressor_name, y_train, y_train_predict, y_test, y_test_predict, Mean, Std):

    
    plt.figure(figsize = (14,6))
    plt.rc('xtick',labelsize=14)
    plt.rc('ytick',labelsize=14)
    '''
    plot the predicted values against train data, calculate accuracy metrics
    '''
    # first subplot - training set
    ax1 = plt.subplot(121)
    
    # plot predicted values vs actual values
    plt.scatter(y_train,y_train_predict)
    
    # add a dash line indicating perfect predictions (Y=X)
    plt.plot([y_train.min(),y_train.max()],[y_train.min(),y_train.max()],'r--',lw=3)
    
    plt.xlabel('True Value ()',fontsize=16)
    plt.ylabel('Predicted Value ()',fontsize=16)
    
    # calculate mean squared error
    mse = lambda y_train,y_train_predict: np.mean((y_train-y_train_predict)**2)
    rmse = mse(y_train,y_train_predict)**0.5
    
    # calculate R2
    R2 = metrics.r2_score(y_train, y_train_predict)
    
    
    plt.fill_between(y_train, Mean-1.96*Std, 
                 Mean+1.96*Std,alpha=0.4, color="c", 
                 label="95% Confidence Interval")

    plt.title(regressor_name+' training set'+'\nRMSE: '+str(rmse)+'\nR2: '+str(R2),fontsize=16)

    plt.rc('xtick',labelsize=14)
    plt.rc('ytick',labelsize=14)
    
    '''
    Do the same but for the test set
    '''
    # second subplot - test set
    ax2 = plt.subplot(122)
    
    # plot predicted values vs actual values
    plt.scatter(y_test,y_test_predict)
    
    # add a dash line indicating perfect predictions (Y=X)
    plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()],'k--',lw=3)

    plt.xlabel('True Value ()',fontsize=16)
    plt.ylabel('Predicted Value ()',fontsize=16)
    
    # calculate mean squared error
    mse = lambda x,y: np.mean((x-y)**2)
    rmse = mse(y_test,y_test_predict)**0.5
    
    # calculate R2
    R2 = metrics.r2_score(y_test,y_test_predict)
    
    plt.title(regressor_name+' test set'+'\nRMSE: '+str(rmse)+'\nR2: '+str(R2),fontsize=16)


    
    plt.show()

    # return mse and mape - we can track the errors across different algorithms
    return rmse, R2


Training an ensamble of NNs. This NN ensamble regressor was also prepared by Hippalgaonkar et al., 2021. The extrapolative performance of this model is of importance, since the quality of prediction around the boundary of the data is crucial. The maximum of the prediction occurs at the boundary.

In [ ]:
# Neural Ensemble Regressor fitted on the whole dataset
ner = NeuralEnsembleRegressor(ensemble_size=10, max_iter=4000)
ner.fit(X_train,y_train)

# save the model to disk
filename = 'NNE_model_new.sav'
pickle.dump(ner, open(filename, 'wb'))

Defining the prediction oracle

In [ ]:

# load the model from disk

ner = pickle.load(open('NNE_model_new.sav', 'rb'))

def prediction_neg(X_norm):  #negated output needed for the minimizing surrogates 
 
    # need to reshape for prediction
    X_norm = np.array(X_norm)
    X_norm = X_norm.reshape(-1,len(columns))

    # the oracle does the prediction
    pred = ner.predict(X_norm)

    # need to return a scalar
    # negative value is returned since by default the optimizer minimizes the function
    return -pred[0]

def prediction(X_norm):   
    X_norm = np.array(X_norm)
    X_norm = X_norm.reshape(-1,len(columns))
    pred = ner.predict(X_norm)
    return pred[0]

Defining the bounds within which the search will be conducted

In [ ]:
min_values = []
max_values = []

for i in range(len(columns)):
    min_values.append(X_train.iloc[:, i].min())
    max_values.append(X_train.iloc[:, i].max())
    
bounds = np.column_stack((min_values, max_values))
bounds

## Comparison of surrogates

Defining the Bayesian loop for BNN surrogate. The BNN is trained several times on the same small data set, each time it is trained, the prediction of the target value is calculated few times. The target value is predicted on a large grid of random points within the bounds. Ideally a regular grid would be used instead of the random points, however a regular grid in the high-dimentional space of any meaningful size takes up too much memory. The mean and standard deviation of the predictions are calculated and using those the acquisition function picks out the best point. 

Random points ensure the space is sampled representativelly, however the density of the points is very small. To increase the density of points on which we predict the target, the best point so far is picked and new grid of smaller size is defined around it. The new grid will consist of the same number of points but since the grid size is smaller, the density of points increases and with it also the probability to find a point with better exploration-exploitation trade-off. 



In [ ]:
from BNN_acq import acquisition

def BNN_BO(X_train, y_train, bounds, iterations_predict = 10, iterations_acq = 10, xi = 0.01):
    y_iter=[]
    
    for _ in range(iterations_acq):

        grid = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(1000000, bounds.shape[0])) #large grid of numbers for which EI will be evaluated
        grid = torch.tensor(grid,  dtype = dtype, device = device)

        best_point = acquisition.expected_improvement_disc(grid, X_train, y_train, iterations_predict, xi) #find the index of the point with the largest EI

        best_acq_tt =  grid[best_point]
        best_acq = best_acq_tt.cpu()# point with the largest EI is selected
        
        #selecting a smaller region centered arounds the best point 
        lower_bound = best_acq - 0.1 # range of features is about 5, this decreases the range to 0.2
        upper_bound = best_acq + 0.1

        grid = np.random.uniform(lower_bound, upper_bound, size=(1000000, bounds.shape[0])) #building a new grid around the point with highest EI
        grid = torch.cat((torch.tensor(grid, dtype=dtype, device=device),torch.unsqueeze(best_acq_tt.flatten(), dim=0)), dim=0)
        
        best_point = acquisition.expected_improvement_disc(grid, X_train, y_train, iterations_predict, xi) #find the index of the point with the largest EI

        best_acq_tt =  grid[best_point]
        best_acq = best_acq_tt.cpu()# point with the largest EI is selected
        
        #selecting a smaller region centered arounds the best point 
        lower_bound = best_acq - 0.01 # this decreases the range to 0.02
        upper_bound = best_acq + 0.01

        grid = np.random.uniform(lower_bound, upper_bound, size=(1000000, bounds.shape[0])) #building a new grid around the point with highest EI
        grid = torch.cat((torch.tensor(grid, dtype=dtype, device=device),torch.unsqueeze(best_acq_tt.flatten(), dim=0)), dim=0)

        best_point = acquisition.expected_improvement_disc(grid, X_train, y_train, iterations_predict, xi) #find the index of the point with the largest EI

        best_acq = grid[best_point] # point with the largest EI is selected
        best_acq = torch.unsqueeze(best_acq.flatten(), dim=0)
        
        # Concatenate the tensors along the first dimension (rows)
        X_train = torch.cat((X_train, best_acq), dim=0)
        y_train = torch.cat((y_train, torch.tensor([prediction(best_acq.cpu().numpy())], dtype = dtype, device = device)),dim=0) # evaluation of the new point with prediction oracle and adding it to training data

        #print(y_train[-1])
        y_iter.append(y_train[-1].tolist())
    return y_iter

Performing the optimization on randomly selected initial points from within the bounds. The same initial points are used for all different surrogate methods. The optimization is repeated several times, each time with a different selection of starting points.

In [ ]:
n_ensemble = 3
n_calls = 30
n_dims = len(columns)

matern_tunable = ConstantKernel(1.0, (1e-12, 1e12)) * Matern(length_scale=np.ones(n_dims), length_scale_bounds=[(1e-12, 1e12)] * n_dims, nu=2.5)
gpregressor = GaussianProcessRegressor(kernel=matern_tunable, n_restarts_optimizer=10, alpha=1e-4, normalize_y=True, noise='gaussian')
nneregressor = NeuralEnsembleRegressor(ensemble_size=10)

res_gp = []
res_dummy = []
res_forest= []
res_gbrt = []
res_nn = []
res_bnn = []


for i in range(n_ensemble):

    N_random_starts = 10
    # Generate N random input points within the bounds
    X_random = []
    y_random = []
    for _ in range(N_random_starts):
        random_array = np.random.uniform(bounds[:, 0], bounds[:, 1])
        X_random.append(random_array)
        y = prediction(random_array)
        y_random.append(y)
    X_random = np.array(X_random).tolist()
    y_random_tt = torch.tensor(np.array(y_random), dtype = dtype, device=device).flatten()    
    X_random_tt = torch.tensor(np.array(X_random), dtype = dtype, device=device)
    
    res_dummy.append(dummy_minimize(prediction_neg, bounds, n_calls = n_calls, x0 = X_random, y0 = y_random))
    res_forest.append(forest_minimize(prediction_neg, bounds, x0 = X_random, y0 = y_random, base_estimator='RF', acq_func='EI', n_calls = n_calls, xi=1.2))
    res_gbrt.append(gbrt_minimize(prediction_neg, bounds,acq_func='EI', n_calls = n_calls,x0 = X_random, y0 = y_random))    
    res_gp.append(gp_minimize(prediction_neg, bounds, n_calls = n_calls, base_estimator = gpregressor, acq_func='LCB', x0 = X_random, y0 = y_random, kappa=1.4))
    res_nn.append(base_minimize(prediction_neg, bounds, n_calls = n_calls, base_estimator=nneregressor, acq_func='EI',x0 = X_random, y0 = y_random, acq_optimizer="sampling", xi=0.01))    
    
    y_iter = BNN_BO(X_random_tt, y_random_tt, bounds, 5, n_calls, 0.01) 
    res_bnn.append(y_iter)

In [ ]:
# List of model names
res_names = ['Dummy', 'Forest', 'Gradient Boosting', 'Gaussian Process', 'Neural Ensemble']
res_array = [res_dummy, res_forest, res_gbrt, res_gp, res_nn]
# Create an empty list to store legends for each line
legends = []
plt.figure(figsize=(10, 6),dpi=300) 
# Loop through the results and plot each line with a label
for i, res_data in enumerate(res_array):
    
    df_res = pd.DataFrame({str(j): res_data[j].func_vals for j in range(len(res_data))})
    df_res = -df_res.mean(axis=1)
    final_df = df_res#.cummax() 
    plt.plot(range(40), final_df, label=res_names[i])  # Use the name from res_names
    legends.append(res_names[i])

# Plot other data (you may need to adjust this part according to your needs)
df = pd.DataFrame(res_bnn).T
df_res = -pd.DataFrame({str(j): res_forest[j].func_vals for j in range(len(res_forest))})

df_mean = df.mean(axis=1)
final_df = df_mean#.cummax()
init_points = df_res[:10].mean(axis=1)#.cummax()
df_iter = pd.concat([init_points, final_df], axis=0)

# Plot the combined data
plt.plot(range(40), df_iter, label="pBNN", color='c')

# Add a legend to the plot
plt.legend(legends + ['pBNN'])
plt.title('Comparing surrogates with 10 random startig points (bounded), averaged over 3 repetitions')
plt.xlabel('Acquisition')
plt.ylabel('Power Factor')
# Show the plot
plt.show()


# Conclusion

The comparison of the point acquisition by different surrogates shows a clear distinction between the pBNN developed in this work and the other surrogate methods. Using pBNN, the early acquisitions, within 30 iterations (corresponding to 30 new theoretical materials) have substentially higher values of the power factor. This means the pBNN is able to easily locate local maxima, with only few points to learn from.

Additional tests show that with an increasing size of initial point set, the pBNN is able to find higher values within the first acquisition. The next best alternative, gaussian process, stops being a viable option after a few hundered initial points on such highdimentional data as it requires too much memory. 

At larger iterations of the acquisition process (100 iterations), the gaussian process performs best as it is better at identifying the absolute maximum. This implies it would be best to combine the the pBNN with the gaussian process in a workflow where many iterations of the acquisition can be performed. The pBNN would locate several local maxima which could be used as the inital data set for the gaussian process to obtain the global maximum.





# Appendix

### pBNN point acquisition from a discrete data set

In [ ]:
# selecting a representative initial data set by clustering the data

from sklearn.cluster import KMeans

X_train_np = X_train.numpy()
y_train_np = y_train.numpy()

# Determine the number of clusters
n_clusters = 5

# Apply K-means clustering
#To ensure diverse clusters, you can use techniques such as K-means++ initialization or specify the "init" parameter in scikit-learn's K-means implementation to obtain diverse initial centroids. 
#Additionally, you can consider using the "n_init" parameter to perform multiple random initializations and select the clustering result with the lowest inertia (sum of squared distances within clusters) to ensure a good solution.

kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10)

kmeans.fit(X_train_np)

# Obtain cluster labels for the data points
cluster_labels = kmeans.labels_

# Access cluster centroids
cluster_centers = kmeans.cluster_centers_

# Select one point from the center of each cluster
initial_X = []
initial_y = []

N = 1 #adds N x n_cluster extra random points

for i in range(n_clusters):
    cluster_X = X_train_np[cluster_labels == i]
    cluster_Y = y_train_np[cluster_labels == i]

    # Randomly select N points from the cluster
    selected_indices = np.random.choice(len(cluster_X), size=N, replace=False)
    selected_X = cluster_X[selected_indices]
    selected_Y = cluster_Y[selected_indices]

    initial_X.extend(selected_X)
    initial_y.extend(selected_Y)

    # Find the center of each cluster to take as the initial starting point - most representative sample of the data
    cluster_center = cluster_centers[i]
    closest_point_index = np.argmin(np.linalg.norm(cluster_X - cluster_center, axis=1))
    initial_X.append(cluster_X[closest_point_index])
    initial_y.append(cluster_Y[closest_point_index])
  
    
initial_X = torch.tensor(np.array(initial_X), dtype = dtype)
initial_y = torch.tensor(np.array(initial_y), dtype = dtype)

In [ ]:
def BNN_BO(X_train, y_train, bounds, iterations_predict = 10, iterations_acq = 10, xi = 0.01):
    y_iter=[]
    
    for _ in range(iterations_acq):

        grid = X_train_np #large grid of numbers for which EI will be evaluated
        best_point = acquisition.expected_improvement_disc(grid, X_train, y_train, iterations_predict, xi) #find the index of the point with the largest EI

        best_acq = grid[best_point] # point with the largest EI is selected
        best_acq = torch.unsqueeze(best_acq.flatten(), dim=0)
        
        # Concatenate the tensors along the first dimension (rows)
        X_train = torch.cat((X_train, best_acq), dim=0)
        y_train = torch.cat((y_train, torch.tensor([prediction(best_acq.cpu().numpy())], dtype = dtype, device = device)),dim=0) # evaluation of the new point with prediction oracle and adding it to training data

        #print(y_train[-1])
        y_iter.append(y_train[-1].tolist())
    return y_iter

In [ ]:
ce_loss = nn.CrossEntropyLoss()
mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 1

In [ ]:
from scipy.stats import norm
def ei_acquisition(mean, std, best_value, xi=0.01):
    z = (mean - best_value-xi) / std
    ei = (mean - best_value) * norm.cdf(z) + std * norm.pdf(z)
    best_index = np.argmax(ei)
    best_point = best_index.item()
    return best_point#, best_index

def lcb_acquisition(mean, std, kappa):
    lcb = mean - kappa * std
    best_index = np.argmin(lcb)
    best_point = best_index.item()
    return best_point

In [ ]:
#define the Bayesian optimization loop

def BNN_BO(X_init, y_init, iterations_train = 1000, iterations_predict = 15, iterations_acq = 10):
    y_iter=[]
    X_iter=[]
    for _ in range(iterations_acq):
        
        model_bnn = nn.Sequential(
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.01, in_features=len(X_train[0]), out_features=400),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.01, in_features=400, out_features=500),
        nn.ReLU(),
        bnn.BayesLinear(prior_mu=0, prior_sigma=0.01, in_features=500, out_features=1),
        nn.Flatten(0,1)
        )
        y_grid = []
        optimizer = torch.optim.RMSprop(model_bnn.parameters(), lr=0.0004)
      
        for _ in range(int(iterations_predict)): # retrain the model multiple times on the same dataset
            
            #train the model on the few initial points 
            for step in range(iterations_train):
                    pre = model_bnn(X_init)
                    mse = mse_loss(pre, y_init)
                    kl = kl_loss(model_bnn)
                    cost = mse + kl_weight*kl

                    optimizer.zero_grad()
                    cost.backward()
                    optimizer.step()
            print('- MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()))

            # Using the trained model we can predict the magnetization for the whole data set

            # Perform multiple iterations to obtain set of predictions
            for _ in range(10):
                y_predictions = model_bnn(X_train0)#pass the whole training data set to the model to obtain a prediciton of y_train0

                # Reshape the predictions tensor to match the grid structure
                y_predictions = y_predictions.view(len(X_train0), -1)

                # Convert the predictions tensor to a nested list
                y_predictions = y_predictions.tolist()

                # Append the predictions to the list
                y_grid.append(y_predictions)
        
        # Convert the list of predictions to a PyTorch tensor
        predictions_tensor = torch.tensor(y_grid)

        # Calculate the mean and standard deviation along the iterations axis
        mean = predictions_tensor.mean(dim=0)
        std = predictions_tensor.std(dim=0)
        
        # use the acquisition function to predict the best point to sample from the data set
        best_point= ei_acquisition(mean, std, y_init.max())
        best_X = torch.unsqueeze(X_train0[best_point], dim=0) #X value at the best acquisition point 
        best_y = torch.unsqueeze(y_train0[best_point], dim=0) #y value at the best acquisition point 

        
        # Make a plot of the acquisition process 
        sorted_indices = torch.argsort(X_train0[:, 0]) #sort the indecies for plotting of 1 feature against magnetization
        sorted_X1 = X_train0[:, 0][sorted_indices]
        sorted_mean = mean[sorted_indices]
        sorted_y = y_train0[sorted_indices]
        sorted_std = std[sorted_indices]
        
        plt.scatter(X_init[:,0], y_init, color = 'green',zorder=2, label = 'initial points + old acq')
        plt.scatter(best_X[:,0], best_y, color = 'red', zorder=3, label = 'current acq')
        plt.plot(sorted_X1, sorted_mean, color = 'blue', zorder=1, label= 'mean')
        
        lower_bound = sorted_mean - 1.96 * sorted_std  # 1.96 is the z-score for a 95% confidence interval
        upper_bound = sorted_mean + 1.96 * sorted_std
        
        # Fill the area between the upper and lower bounds with a translucent color
        plt.fill_between(sorted_X1, lower_bound.flatten(), upper_bound.flatten(), color='lightgreen', alpha=0.9,zorder=0, label='95% confidance interval')

        plt.legend()
        plt.xlabel('Transformed mean Effective Coordination')
        plt.ylabel('Transformed Magnetization')
        plt.show()
        
        X_init = torch.cat((X_init, best_X), dim=0)
        y_init = torch.cat((y_init, best_y),dim=0)
        print(best_y)
        y_iter.append(y_init[-1].tolist())
        X_iter.append(X_init[-1].tolist())
        
        
    return y_iter, X_iter

### Hyperparameter optimization 

For the sklearn surrogates the hyperparameters have been optimized such as what follows:


In [ ]:
import optuna
from sklearn import metrics
def objective(trial):
    acq = []
    #x = trial.suggest_float('x', 1.1, 1.8)
    #y = trial.suggest_float('y', 0.1, 0.4, log=True)
    y = trial.suggest_float('y', 0.05, 0.25)
    n_calls = 30

    # Number of repetitions to get the optimization statistics.
    n_ensemble = 3

    n_dims = len(columns)

    matern_tunable = ConstantKernel(1, (1e-12, 1e12)) * Matern(
        length_scale=np.ones(n_dims), length_scale_bounds=[(1e-12, 1e12)] * n_dims, nu=2.5)


    res_gp = []
    for i in range(n_ensemble):
        
        gpregressor = GaussianProcessRegressor(kernel=matern_tunable, n_restarts_optimizer=10, 
                                            alpha=y, normalize_y=True, noise='gaussian')
        
        res_gp.append(gp_minimize(prediction, 
                        bounds,
                        n_calls = n_calls, 
                        base_estimator=gpregressor,
                        acq_func='LCB', 
                        #random_state = 1, leave random state seed random, since ensamble is used, each hyperparameter selection is averaged over 3 trails - more representative 
                        n_random_starts=1,
                        #xi=0.01
                        kappa=1.35))

        #Transforms back to original values
        X_reshape = np.array(res_gp[i].x).reshape(-1,len(columns))
        X_original = pipeline.inverse_transform(X_reshape)
    res = res_gp
    method = ['Gaussian']
#code from downstairs put thru gpt to make more efficient during iteretive hyperparameter opt. 40% decrease in comp time
    df_res = pd.DataFrame({str(i): res[i].func_vals for i in range(len(res))}) 
    df_min = pd.DataFrame({str(i): np.min(df_res.iloc[:j+1, i]) for i in range(df_res.shape[1])} for j in range(len(df_res)))
    df_power = pd.DataFrame(-df_min.min(axis=1), columns=[method])
    return -df_power.max()

'''
list_1 = np.linspace(0.001, 0.1, 100)
list_2 = np.log(np.linspace(1.00001,1.1 , 1000000)) #adjust to log wise sampling - samples log dist.
search_space = {
    #'x': list_1,
    'y': list_2,
}
'''

study_gp= optuna.create_study()
study_gp.optimize(objective, n_trials=30)

#study_gp = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space))
#study_gp.optimize(objective, n_trials=100)

In [ ]:
hyperparameter_x = []
hyperparameter_y = []
objective_values = []

for trial in study_gp.trials:
    hyperparameter_x.append(trial.params['x'])
    hyperparameter_y.append(trial.params['y'])
    objective_values.append(trial.value)
'''  
x1 = []
y1 = []
x2 = []
y2 = []
for trial in study_gp.trials:
    if trial.params['y'] == 1.5:
        x1.append(trial.params['x'])
        y1.append(trial.value)
    else: 
        x2.append(trial.params['x'])
        y2.append(trial.value)
'''  

# Plot the hyperparameter values against the objective values
plt.scatter(hyperparameter_y, objective_values, label='Objective Values')
plt.scatter(x1, y1, label='nu = 1.5')
plt.scatter(x2, y2, label='nu = 2.5')
plt.xlabel('alpha')  # Replace with the name of the hyperparameter
plt.ylabel('Objective Value')
plt.title('Hyperparameter Optimization - GP')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

hyperparameter_x1 = []
hyperparameter_x2 = []
objective_values = []

for trial in study_gp.trials:
    #hyperparameter_x1.append(trial.params['x'])
    hyperparameter_x2.append(trial.params['y'])
    objective_values.append(trial.value)

def moving_average(data, window_size):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

def plot(hyperparameter):
    
    x = np.array(hyperparameter)
    y = np.array(objective_values)

    # Get indices that would sort the x array
    sorted_indices = np.argsort(x, kind='mergesort')

    # Sort both arrays based on the sorted indices
    sorted_x = x[sorted_indices]
    sorted_y = y[sorted_indices]

    # Calculate moving average
    window_size = 5
    moving_avg = moving_average(sorted_y, window_size)

    # Plot scatter plot and moving average
    plt.scatter(x, y, label='Data')
    plt.plot(sorted_x[window_size - 1:], moving_avg, color='red', label='Moving Average')
    plt.xlabel('alpha')
    plt.ylabel('Y')
    plt.legend()
    plt.title('Scatter plot with Moving Average - random seed, ensemble = 3')
    # Set x-axis to logarithmic scale
    plt.xscale('log')
    return plt.show()

plot(hyperparameter_x2)

### Plotting the minimization process projected on a 2D plane:

In [ ]:
n_dims = len(bounds)

matern_tunable = ConstantKernel(1.0, (1e-12, 1e12)) * Matern(
    length_scale=np.ones(n_dims), length_scale_bounds=[(1e-12, 1e12)] * n_dims, nu=2.5)

gpregressor = GaussianProcessRegressor(kernel=matern_tunable, n_restarts_optimizer=10, 
                                       alpha=0.06, normalize_y=True, noise='gaussian')

res = gp_minimize(prediction, 
                  bounds,
                  n_calls=100, 
                  base_estimator=gpregressor,
                  acq_func='LCB',
                  n_random_starts=10, 
                  kappa=1.4)

# Extract the iterations and corresponding x values
iterations = res.x_iters
x_values = np.array(iterations)

In [ ]:
#computing the grid
twoD = [np.linspace(bounds[i, 0], bounds[i, -1], 100) for i in [0,1]]
X_grid = np.meshgrid(*twoD)
result_grid = np.empty_like(X_grid[0])

# Iterate over the indices of X_grid arrays
for i in range(X_grid[0].shape[0]):
    for j in range(X_grid[0].shape[1]):
        # Get the paired elements from X_grid arrays
        x = np.array([X_grid[0][i, j], X_grid[1][i, j]])
        
        # Reshape the x array to a 2D array with one sample and two features
        x = x.reshape(1, -1)
        numbers_to_append = np.array([x_values[-1, i] for i in range(2, 22)]) # Adding the numbers corresponding to the other dimentions 

        x = np.append(x, [numbers_to_append], axis=1)
        # Call ner_pca.predict with the reshaped input
        result = ner.predict(x)
        
        # Store the result in the corresponding position of result_grid
        result_grid[i, j] = result

In [ ]:
# Plot the function and the iterations
plt.figure(figsize=(8, 6))
contour = plt.contourf(twoD[0], twoD[1], result_grid, levels=100, cmap='jet')
plt.colorbar()
plt.scatter(x_values[-1, 0], x_values[-1, 1], c='red', linewidths=3, alpha=1, marker ='x')
plt.plot(x_values[:, 0], x_values[:, 1], '-o', markersize=4, linewidth=1, color='black')

for i in range(len(x_values)):
    alpha = (i + 1) / len(x_values)  # Scale alpha based on the order of iteration
    plt.plot(x_values[i,0], x_values[i,1], 'wo', markersize=3, alpha=alpha)
plt.xlabel('Atomic weight')
plt.ylabel('Covalent radius')
plt.title('Minimization using gp_minimize')
plt.show()

## references

http://krasserm.github.io/2018/03/21/bayesian-optimization/

http://krasserm.github.io/2019/03/14/bayesian-neural-networks/

https://github.com/Harry24k/bayesian-neural-network-pytorch/tree/master

https://towardsdatascience.com/why-you-should-use-bayesian-neural-network-aaf76732c150#:~:text=What%20is%20Bayesian%20Neural%20Network,that%20best%20fit%20the%20data

https://www.authorea.com/doi/full/10.22541/au.162679475.54895263/v1

http://krasserm.github.io/2018/03/19/gaussian-processes/